<a href="https://colab.research.google.com/github/Nicolas-BZRD/Multi-Video-Temporal-Synchronization/blob/main/Video_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notebook to create a video dataset with delay.

In [1]:
import os
from random import randint

import cv2
import numpy as np

from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Processing functions

Tools functions

In [3]:
#Convert video to an array with shape(numberFrame, height, width, channel).
def videoToArray(path):
  video = cv2.VideoCapture(path)
  fps = video.get(5)
  arr = []

  success, image = video.read()

  while success:
    arr.append(image)
    success, image = video.read()

  return arr, fps

In [4]:
#Reduce the number of images in the video.
def reduceFps(arr, fps, desiredFPS):
  step = int(fps/desiredFPS)
  arrFPS = arr[0::step]
  return arrFPS, desiredFPS

In [5]:
#Convert RGB images in gray scales images.
def RGBtoGray(arr):
  rgb_weights = [0.2989, 0.5870, 0.1140]
  arrGray = []

  for img in arr:
    arrGray.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.uint8))
  return arrGray

In [6]:
#Resize the video aspect ratio.
def changeRatio(arr, width, height):
  arrResize = []

  for img in arr:
    arrResize.append(cv2.resize(img, dsize=(width, height), interpolation=cv2.INTER_CUBIC))

  return arrResize

In [7]:
#Split the video into several samples
def splitVideo(arr, stepSamplingFrame):
  return [arr[i : i + stepSamplingFrame] for i in range(0, len(arr), stepSamplingFrame)]

In [8]:
#Outputs two videos spaced out of x frames. Take two separate videos or just one that will be split
def delayBetweenTwoArrays(fpsDelay, arrayFirstVideo, arraySecondVideo=[]):
    arrDelay = arrayFirstVideo[fpsDelay:]
    if(arraySecondVideo != []):
      arrAdvance = arraySecondVideo[:-fpsDelay]
    else:
      arrAdvance = arrayFirstVideo[:-fpsDelay]
      
    return arrDelay, arrAdvance

In [9]:
#Convert the array video to an video file saved on disk.
def arrayToVideo(arr, name, height, width, fps, gray):
  if(gray):
    out = cv2.VideoWriter(f"{name}.mp4", cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), fps, (width, height), 0)
  else:
    out = cv2.VideoWriter(f"{name}.mp4", cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), fps, (width, height))
  
  [out.write(f) for f in arr]
  out.release()

Function to apply all differents tools:

In [10]:
def videoProcessingYoutube(directory, saveDirectory, height=240, width=240, sampleTime=2, desiredFps=10, gray=True):
  id = 0
  for filename in tqdm(os.listdir(directory)):
    file = os.path.join(directory, filename)
    if os.path.isfile(file):
      #1 - convert video
      arr, fps =  videoToArray(file)

      #2 - reduce fps
      if fps>desiredFps:
        arr, fps = reduceFps(arr, fps, desiredFps)
      else: fps = int(fps)

      #3 - Transform video in gray scales video
      if(gray):
        arr = RGBtoGray(arr)

      #4 - Change ratio of the video
      arr = changeRatio(arr, height, width)

      #5 - Create a small video sample of 2s
      arr = splitVideo(arr, sampleTime*fps)

      for sample in arr:
        fpsDelay = randint(0, max(int(len(sample)/2)-1, 0))
        arrDelayed, arrAdvanced = delayBetweenTwoArrays(fpsDelay, sample)

        #7 - Save modified video
        arrayToVideo(arrDelayed, f"{saveDirectory}/{id}arrD_{fpsDelay}", height, width, fps, gray)
        arrayToVideo(arrAdvanced, f"{saveDirectory}/{id}arrA_{fpsDelay}",height, width, fps, gray)
        id +=1
  return id

#Video processing

##Youtube dataset

In [11]:
sizeDataSet = videoProcessingYoutube("/content/drive/MyDrive/ISIA Lab/Dataset/YouTube", "/content/drive/MyDrive/ISIA Lab/Dataset/YouTube processed")

100%|██████████| 1969/1969 [52:58<00:00,  1.61s/it]


In [12]:
print(f"Size of youtube dataset created: {sizeDataSet}")

Size of youtube dataset created: 13636
